# Determine optimal number of clusters

In [1]:
import os
# Set environment variables to disable multithreading
# as users will probably want to set the number of cores
# to the max of their computer.
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"


In [1]:
import time

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

from astroExplain.spectra import clustering
from sdss.metadata import MetaData

meta = MetaData()

# Custom functions

# Data ingestion

In [2]:
bin_id = "bin_03"
phd_dir = "/home/elom/phd"
spectra_dir = f"{phd_dir}/00_phd_code/spectra"
explanations_dir = f"{phd_dir}/00_phd_code/explanations"
paper_figures_dir = f"{phd_dir}/00_paper_explain-me-why/sections/figures/"
wave = np.load(f"{spectra_dir}/wave_spectra_imputed.npy")


# MSE
From previous analysis we determined the number of clusters is 5

In [3]:
score_name = "mse_noRel100"
explanation_run_id = "20250427190650_uniform_5000_scale"

weights = np.load(
    f"{explanations_dir}/{bin_id}/{score_name}/weights_{explanation_run_id}.npy",
)

anomalies_array = np.load(
    f"{spectra_dir}/{bin_id}/{score_name}/top_anomalies.npy",
    mmap_mode="r"
)

anomalies_df = pd.read_csv(
    f"{spectra_dir}/{bin_id}/{score_name}/top_anomalies.csv.gz",
    index_col="specobjid",
)


## Train kmeans

In [4]:
X = clustering.get_weights_per_segments(
    weights=weights, n_segments=128
)
X = np.abs(X)/np.max(np.abs(X), axis=1, keepdims=True)

n_clusters = 5

start_time = time.perf_counter()
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
end_time = time.perf_counter()
print(f"train time: {end_time - start_time:.2f} seconds")

Base size: 29, Residual size: 61
New number of segments: 129
train time: 0.21 seconds


In [5]:
cluster_labels = kmeans.labels_
(
    spectra_cluster_dict, weights_cluster_dict
) = clustering.group_spectra_by_cluster(
    cluster_labels=cluster_labels,
    anomalies_array=anomalies_array,
    weights=weights
)

Cluster: 0, N. spectra: 1993
Cluster: 1, N. spectra: 3704
Cluster: 2, N. spectra: 1603
Cluster: 3, N. spectra: 1759
Cluster: 4, N. spectra: 941


# MSE noRel 97

In [7]:
score_name = "mse_noRel97"
explanation_run_id = "20250427151754_uniform_5000_scale"

weights = np.load(
    f"{explanations_dir}/{bin_id}/{score_name}/weights_{explanation_run_id}.npy",
)


# MSE filter 250 kms noRel100

In [20]:
score_name = "mse_filter_250kms_noRel100"
explanation_run_id = "20250427094150_uniform_5000_scale"

weights = np.load(
    f"{explanations_dir}/{bin_id}/{score_name}/weights_{explanation_run_id}.npy",
)

# MSE filter 250 kms noRel97

In [11]:
score_name = "mse_filter_250kms_noRel97"
explanation_run_id = "20250427105355_uniform_5000_scale"

weights = np.load(
    f"{explanations_dir}/{bin_id}/{score_name}/weights_{explanation_run_id}.npy",
)